# SET Data Analysis

This notebook contains comprehensive analysis of DSC SET (Student Evaluation of Teaching) data, including data cleaning, exploratory data analysis, and statistical modeling.

In [ ]:

# Import all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Statistical and machine learning libraries
from scipy.stats import ttest_ind, f_oneway, chi2_contingency, pearsonr, spearmanr
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

# Machine learning libraries
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve,
    precision_recall_curve, average_precision_score
)
from sklearn.inspection import permutation_importance

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## Data Loading/Cleaning
In this section we load and merge the data for EDA

*Load and clean SETs Data*

In [ ]:
# load sets into dataframe
dsc_sets = pd.read_csv('datasets/dsc_sets.csv')

# merge duplicate columns
dsc_sets["Enrolled Resp Rate"] = dsc_sets["Enrolled/ Resp Rate"].fillna(dsc_sets["Enrolled/  Resp Rate"])
dsc_sets = dsc_sets.drop(columns=["Enrolled/ Resp Rate", "Enrolled/  Resp Rate"])

# rename columns for easier access
dsc_sets.columns = dsc_sets.columns.str.replace('*', '', regex=False)
dsc_sets.columns = dsc_sets.columns.str.replace(' ', '_')
dsc_sets['Term'] = dsc_sets['Term'].str.lower()


# Extract the number before the parentheses
dsc_sets['Enrolled_Resp_Number'] = dsc_sets['Enrolled_Resp_Rate'].str.extract(r'(\d+)\s*\(')[0].astype(int)
# Extract the percentage inside parentheses
dsc_sets['Enrolled_Resp_Pct'] = dsc_sets['Enrolled_Resp_Rate'].str.extract(r'\(([\d\.]+)%\)')[0].astype(float)

# remove Letter from Avg Grade Received
dsc_sets["Avg_Grade_Received"] = (
    dsc_sets["Avg_Grade_Received"]
    .str.extract(r'(\d+\.\d+|\d+)')   # extract first number
    .astype(float)
)

# delete unnecessary columns
dsc_sets = dsc_sets.drop(columns=["Course", "Enrolled_Resp_Rate"])
# remove dsc96 course (doesn't have grades)
dsc_sets = dsc_sets[dsc_sets['course_title'] != 'dsc95']

dsc_sets = dsc_sets.dropna(subset=['Avg_Grade_Received'])
dsc_sets.head()

*Merge SETs data with Utilization Rate from WebReg Data*

In [ ]:
webreg_data = pd.read_csv('webreg_data/results/webreg_processed_data.csv')
webreg_data.head()
merged_data = webreg_data[['course', 'quarter', 'utilization_rate']]
merged_data = merged_data.copy()

#drop underscore from course column to match dsc_sets
merged_data['course'] = merged_data['course'].str.replace('_', '').str.lower()
merged_data = merged_data.rename(columns={'course': 'course_title', 'quarter': 'Term'})
# merge datasets on course title and term
final_data = pd.merge(dsc_sets, merged_data, on=['course_title', 'Term'], how='inner')
cols = ['course_title'] + [c for c in final_data.columns if c != 'course_title']
final_data = final_data[cols]

final_data.head()

## EDA of SETs + Utilization Data

### Correlation of Numeric Course Metrics
This visualization shows the pairwise linear correlations among key numeric course metrics in the dataset, focusing on the lower triangle of the correlation matrix to avoid redundancy.

The variables included are:
* Avg_Grade_Received – average grade students earned in the course
* Avg_Hours_Worked – average weekly hours students reported spending on the course
* Learning_Average – students’ evaluation of how much they learned
* Structure_Average – students’ evaluation of the course’s organization and clarity
* Environment_Average – students’ evaluation of the classroom and learning environment
* Enrolled_Resp_Number – number of students enrolled/responded
* Enrolled_Resp_Pct – enrollment/response percentage

`Structure_Average`, `Learning_Average`, and `Environment_Average` exhibit strong positive correlations, indicating consistent student perceptions across these aspects.

In [ ]:
numeric_cols = ['Avg_Grade_Received', 'Avg_Hours_Worked',
                'Learning_Average', 'Structure_Average', 
                'Environment_Average', 'Enrolled_Resp_Number',
                'Enrolled_Resp_Pct']
numeric_df = final_data[numeric_cols]

In [ ]:
corr = numeric_df.corr()
plt.figure(figsize=(8,6))  # set figure size
mask = np.triu(np.ones_like(corr, dtype=bool))

sns.heatmap(
    corr,
    annot=True,       # show correlation values
    fmt=".2f",        # format numbers
    cmap='coolwarm',  # color map
    vmin=-1, vmax=1,   # fix color scale
    mask=mask      # hide diagonal
)
plt.title("Correlation Between Numeric Metrics")
plt.show()

### Test 1: Test for Multi-Colinearity

**Purpose**: Determine if numeric predictors are highly correlated, which can inflate regression coefficients and make them unstable.

**Testing**: 
* **Coefficient**: Change in `utilization_rate` for a one-unit increase in predictor.
* **p-value**: If predictor's effect is significantly associated with `utilization_rate` (is if \< 0.05)
* **$R^2$**: Proportion of variance in `utilization_rate` explained by the predictor alone.


In [ ]:
X = numeric_df.dropna() 
X = sm.add_constant(X) 
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_data

### Test 2: Simple Linear Regression of Numeric Predictors on Utilization Rate

**Purpose**: Find if any numeric variables predict `utilization_rate` in a univariate model.

**Testing**: 
* **Coefficient**: Change in `utilization_rate` for a one-unit increase in predictor.
* **p-value**: If predictor's effect is significantly associated with `utilization_rate` (is if \< 0.05)
* **$R^2$**: Proportion of variance in `utilization_rate` explained by the predictor alone.


In [ ]:
results = []

for col in numeric_cols:
    X = final_data[[col]]
    Y = final_data['utilization_rate']
    
    # Add constant for intercept
    X = sm.add_constant(X)
    
    # Fit model
    model = sm.OLS(Y, X).fit()
    
    # Save results
    results.append({
        'Predictor': col,
        'Coefficient': model.params[col],
        'Intercept': model.params['const'],
        'R_squared': model.rsquared,
        'p_value': model.pvalues[col]
    })

# Convert to DataFrame for easy viewing
regression_results = pd.DataFrame(results)
regression_results

**Interpretation**: 
* **Coefficient**: Change in `utilization_rate` for a one-unit increase in predictor.
* **p-value**: If predictor's effect is significantly associated with `utilization_rate` (is if \< 0.05)
* **$R^2$**: Proportion of variance in `utilization_rate` explained by the predictor alone.


---

## Advanced Analysis: Difficulty Prediction and Statistical Testing

In [ ]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Statistical and machine learning libraries
from scipy.stats import ttest_ind, f_oneway, chi2_contingency, pearsonr, spearmanr
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import statsmodels.api as sm

# Machine learning libraries
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve,
    precision_recall_curve, average_precision_score
)
from sklearn.inspection import permutation_importance

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
# Load SET data (ONLY data source - no WebReg)
dsc_sets = pd.read_csv('datasets/dsc_sets.csv')

## Step 2: Data Cleaning and Feature Engineering

We need to:
1. Extract course numbers from course titles
2. Standardize term formats
3. Clean and extract numeric values from grade fields
4. Aggregate SET data by course and quarter
5. Create derived features for analysis

**Note:** This analysis uses ONLY SET data. No WebReg data is used.

In [ ]:
# Helper function to extract course number from course title
def extract_course_number(course_title):
    """Extract numeric course number from course title (e.g., 'dsc100' -> 100)"""
    if pd.isna(course_title):
        return None
    import re
    match = re.search(r'dsc(\d+)', str(course_title).lower())
    if match:
        return int(match.group(1))
    return None

# Helper function to standardize term format
def standardize_term(term):
    """Convert term format to match webreg format (e.g., 'FA24' -> 'fa24')"""
    if pd.isna(term):
        return None
    term_str = str(term).upper()
    term_map = {
        'FA24': 'fa24', 'WI25': 'wi25', 'SP25': 'sp25',
        'FALL 2024': 'fa24', 'WINTER 2025': 'wi25', 'SPRING 2025': 'sp25',
        'FA 2024': 'fa24', 'WI 2025': 'wi25', 'SP 2025': 'sp25'
    }
    if term_str in term_map:
        return term_map[term_str]
    import re
    match = re.search(r'(FA|WI|SP|FALL|WINTER|SPRING)\s*(\d{2,4})', term_str)
    if match:
        season = match.group(1)[:2].upper()
        year = match.group(2)[-2:]
        if season.startswith('FA'):
            return f'fa{year}'
        elif season.startswith('WI'):
            return f'wi{year}'
        elif season.startswith('SP'):
            return f'sp{year}'
    return None

# Clean SET data (ONLY data source)

# First, clean column names (before creating new columns)
dsc_sets.columns = dsc_sets.columns.str.replace('*', '', regex=False)
dsc_sets.columns = dsc_sets.columns.str.replace(' ', '_')
dsc_sets.columns = dsc_sets.columns.str.replace('/', '_')

# Handle Enrolled Resp Rate columns (merge duplicates)
if "Enrolled_Resp_Rate" in dsc_sets.columns and "Enrolled__Resp_Rate" in dsc_sets.columns:
    dsc_sets["Enrolled_Resp_Rate"] = dsc_sets["Enrolled_Resp_Rate"].fillna(dsc_sets["Enrolled__Resp_Rate"])
    dsc_sets = dsc_sets.drop(columns=["Enrolled__Resp_Rate"], errors='ignore')
elif "Enrolled_Resp_Rate" in dsc_sets.columns:
    pass  # Already exists
elif "Enrolled__Resp_Rate" in dsc_sets.columns:
    dsc_sets["Enrolled_Resp_Rate"] = dsc_sets["Enrolled__Resp_Rate"]
    dsc_sets = dsc_sets.drop(columns=["Enrolled__Resp_Rate"], errors='ignore')

# Drop unnecessary columns first
dsc_sets = dsc_sets.drop(columns=["Course"], errors='ignore')

# Handle duplicate columns BEFORE converting to numeric
# Find all duplicate column names
all_cols = list(dsc_sets.columns)
duplicate_cols = [col for col in set(all_cols) if all_cols.count(col) > 1]

# For each duplicate column, merge them intelligently (keep non-NaN values)
for col_name in duplicate_cols:
    col_indices = [i for i, c in enumerate(dsc_sets.columns) if c == col_name]
    if len(col_indices) > 1:
        # Get all columns with this name
        cols_data = [dsc_sets.iloc[:, idx] for idx in col_indices]
        
        # Merge: start with first column, fill NaN with values from other columns
        merged_col = cols_data[0].copy()
        for other_col in cols_data[1:]:
            # Fill NaN values with non-NaN values from other column
            mask = merged_col.isna() & other_col.notna()
            merged_col[mask] = other_col[mask]
        
        # Replace first occurrence with merged data
        dsc_sets.iloc[:, col_indices[0]] = merged_col
        
        # Drop all other occurrences
        cols_to_drop = [dsc_sets.columns[i] for i in col_indices[1:]]
        dsc_sets = dsc_sets.drop(columns=cols_to_drop, errors='ignore')
        
        # Rename to ensure consistency (in case pandas kept different names)
        dsc_sets = dsc_sets.rename(columns={dsc_sets.columns[col_indices[0]]: col_name})

# Now remove any remaining duplicates by position
dsc_sets = dsc_sets.loc[:, ~dsc_sets.columns.duplicated(keep='first')]

# Verify duplicates are removed
remaining_duplicates = [col for col in set(dsc_sets.columns) if list(dsc_sets.columns).count(col) > 1]
if remaining_duplicates:
    # Force remove by recreating dataframe with unique columns
    dsc_sets = dsc_sets.iloc[:, ~dsc_sets.columns.duplicated(keep='first')]
else:

# NOW extract numeric grade from Avg_Grade_Received (after duplicates are removed)
if "Avg_Grade_Received" in dsc_sets.columns:
    # Get the column as a Series (should be unique now)
    col_idx = list(dsc_sets.columns).index("Avg_Grade_Received")
    col_series = dsc_sets.iloc[:, col_idx]
    
    # Check if column contains strings (like "3.93 (A-)") or is already numeric
    if col_series.dtype == 'object':
        # Extract numeric GPA value from strings like "3.93 (A-)" or "3.93"
        # Pattern matches: start of string, digits, optional decimal point and more digits
        extracted = col_series.astype(str).str.extract(r'^(\d+\.?\d*)', expand=False)
        dsc_sets.iloc[:, col_idx] = pd.to_numeric(extracted, errors='coerce')
        non_null_count = dsc_sets.iloc[:, col_idx].notna().sum()
    else:
        # Already numeric, just ensure it's the right type
        dsc_sets.iloc[:, col_idx] = pd.to_numeric(col_series, errors='coerce')
else:
    # Try alternative names
    for col in dsc_sets.columns:
        if 'Grade' in col and 'Received' in col:
            col_idx = list(dsc_sets.columns).index(col)
            col_series = dsc_sets.iloc[:, col_idx]
            if col_series.dtype == 'object':
                # Extract numeric values from strings like "3.93 (A-)"
                extracted = col_series.astype(str).str.extract(r'^(\d+\.?\d*)', expand=False)
                dsc_sets.iloc[:, col_idx] = pd.to_numeric(extracted, errors='coerce')
            else:
                dsc_sets.iloc[:, col_idx] = pd.to_numeric(col_series, errors='coerce')
            # Rename to Avg_Grade_Received
            dsc_sets = dsc_sets.rename(columns={col: "Avg_Grade_Received"})
            break

# Extract course number and quarter
dsc_sets['course_number'] = dsc_sets['course_title'].apply(extract_course_number)
dsc_sets = dsc_sets[dsc_sets['course_number'].notna()]
dsc_sets['quarter'] = dsc_sets['Term'].apply(standardize_term)


# Safe function to get non-null count
def safe_nonnull_count(df, col_name):
    """Safely count non-null values in a column"""
    if col_name not in df.columns:
        return None
    try:
        col_idx = list(df.columns).index(col_name)
        col_data = df.iloc[:, col_idx]
        count = col_data.notna().sum()
        return int(count) if not isinstance(count, pd.Series) else int(count.iloc[0]) if len(count) > 0 else 0
    except:
        return None

hours_count = safe_nonnull_count(dsc_sets, 'Avg_Hours_Worked')
grade_count = safe_nonnull_count(dsc_sets, 'Avg_Grade_Received')
learning_count = safe_nonnull_count(dsc_sets, 'Learning_Average')
structure_count = safe_nonnull_count(dsc_sets, 'Structure_Average')
environment_count = safe_nonnull_count(dsc_sets, 'Environment_Average')



## Step 3: Aggregate SET Data and Create Feature Set

We'll aggregate SET data by course and quarter to create our feature set. The dataset will include:
- **SET features**: Learning Average, Structure Average, Environment Average, Study Hours, Grades (ALL features from SET data only)


In [ ]:
# Step 3: Aggregate SET data by course and quarter

# Find the actual column names after cleaning
hours_col_set = None
grade_col_set = 'Avg_Grade_Received'  # We created this
learning_col_set = None
structure_col_set = None
environment_col_set = None

for col in dsc_sets.columns:
    if 'Hours' in col or 'hours' in col.lower():
        hours_col_set = col
    if 'Learning' in col and 'Average' in col:
        learning_col_set = col
    if 'Structure' in col and 'Average' in col:
        structure_col_set = col
    if 'Environment' in col and 'Average' in col:
        environment_col_set = col


# Convert columns to numeric before aggregation (handle any string values)
numeric_cols = []
cols_to_convert = []

if hours_col_set and hours_col_set in dsc_sets.columns:
    cols_to_convert.append(hours_col_set)
if grade_col_set and grade_col_set in dsc_sets.columns:
    cols_to_convert.append(grade_col_set)
if learning_col_set and learning_col_set in dsc_sets.columns:
    cols_to_convert.append(learning_col_set)
if structure_col_set and structure_col_set in dsc_sets.columns:
    cols_to_convert.append(structure_col_set)
if environment_col_set and environment_col_set in dsc_sets.columns:
    cols_to_convert.append(environment_col_set)

# Convert each column to numeric - handle duplicates and ensure Series
for col in cols_to_convert:
    if col in dsc_sets.columns:
        try:
            # Check if column exists and get column index
            col_indices = [i for i, c in enumerate(dsc_sets.columns) if c == col]
            if len(col_indices) > 0:
                # Use iloc to get the first occurrence as a Series
                col_idx = col_indices[0]
                col_series = dsc_sets.iloc[:, col_idx]
                
                # Special handling for grade column (may contain strings like "3.93 (A-)")
                if col == grade_col_set and col_series.dtype == 'object':
                    # Extract numeric GPA from strings like "3.93 (A-)" or "3.93"
                    # Try to extract the first numeric value (GPA)
                    extracted = col_series.astype(str).str.extract(r'^(\d+\.?\d*)', expand=False)
                    dsc_sets.iloc[:, col_idx] = pd.to_numeric(extracted, errors='coerce')
                    non_null_after = dsc_sets.iloc[:, col_idx].notna().sum()
                else:
                    # Convert to numeric (handles already-numeric or other string formats)
                    dsc_sets.iloc[:, col_idx] = pd.to_numeric(col_series, errors='coerce')
                
                numeric_cols.append(col)
        except Exception as e:
            continue


# Build aggregation dictionary (only for numeric columns with actual data)
agg_dict_set = {}
cols_to_check = [
    (hours_col_set, 'study_hours'),
    (grade_col_set, 'avg_grade'),
    (learning_col_set, 'learning_avg'),
    (structure_col_set, 'structure_avg'),
    (environment_col_set, 'environment_avg')
]

for col_name, target_name in cols_to_check:
    if col_name and col_name in numeric_cols:
        # Check if column has any non-null values
        col_idx = list(dsc_sets.columns).index(col_name) if col_name in dsc_sets.columns else None
        if col_idx is not None:
            col_data = dsc_sets.iloc[:, col_idx]
            non_null_count = col_data.notna().sum()
            if non_null_count > 0:
                agg_dict_set[col_name] = 'mean'
            else:


if len(agg_dict_set) > 0:
    # Only aggregate numeric columns
    merged_df = dsc_sets.groupby(['course_number', 'quarter'])[list(agg_dict_set.keys())].mean().reset_index()
    
    # Rename columns
    rename_dict_set = {}
    if hours_col_set and hours_col_set in merged_df.columns:
        rename_dict_set[hours_col_set] = 'study_hours'
    if grade_col_set and grade_col_set in merged_df.columns:
        rename_dict_set[grade_col_set] = 'avg_grade'
    if learning_col_set and learning_col_set in merged_df.columns:
        rename_dict_set[learning_col_set] = 'learning_avg'
    if structure_col_set and structure_col_set in merged_df.columns:
        rename_dict_set[structure_col_set] = 'structure_avg'
    if environment_col_set and environment_col_set in merged_df.columns:
        rename_dict_set[environment_col_set] = 'environment_avg'
    
    merged_df = merged_df.rename(columns=rename_dict_set)
else:
    merged_df = pd.DataFrame(columns=['course_number', 'quarter', 'study_hours', 'avg_grade',
                                     'learning_avg', 'structure_avg', 'environment_avg'])


# Check data completeness with safe column access

def safe_count(df, col_name):
    """Safely count non-null values in a column, handling duplicates"""
    if col_name not in df.columns:
        return None
    try:
        # Get column by index to avoid duplicate issues
        col_indices = [i for i, c in enumerate(df.columns) if c == col_name]
        if len(col_indices) > 0:
            col_idx = col_indices[0]
            col_data = df.iloc[:, col_idx]
            count = col_data.notna().sum()
            # Ensure it's a scalar
            if isinstance(count, pd.Series):
                count = count.iloc[0] if len(count) > 0 else 0
            return int(count)
    except Exception as e:
        return None
    return None

study_hours_count = safe_count(merged_df, 'study_hours')
if study_hours_count is not None:
else:

avg_grade_count = safe_count(merged_df, 'avg_grade')
if avg_grade_count is not None:
else:

learning_count = safe_count(merged_df, 'learning_avg')
if learning_count is not None:
else:

structure_count = safe_count(merged_df, 'structure_avg')
if structure_count is not None:
else:

environment_count = safe_count(merged_df, 'environment_avg')
if environment_count is not None:
else:

merged_df.head()


## Step 4: Manually Define Difficulty Labels (Ground Truth)

**Key Decision: Choosing the Feature for Manual Difficulty Definition**

We will use **Study Hours** (Avg_Hours_Worked) from SET data as our manual difficulty definition because:

1. **Direct Workload Indicator**: Study hours directly reflect the time commitment required, which correlates with difficulty
2. **Objective Measure**: Based on actual reported hours, not subjective ratings
3. **Interpretable**: Higher hours = more difficult/time-consuming course
4. **Available in SET Data**: We have complete SET data with study hours for all courses

**Definition:**
- **Difficult (1)**: Study hours ≥ median (courses requiring more time commitment)
- **Easy (0)**: Study hours < median (courses requiring less time commitment)

This creates a balanced binary classification problem based on workload/difficulty.

**Note:** All features (study hours, learning, structure, environment averages, grades) come from SET data only.


In [ ]:
# Step 4: Manually define difficulty based on multiple SET features

# Filter to rows with all required features
required_features = ['study_hours', 'learning_avg', 'structure_avg', 'environment_avg']
df_with_features = merged_df[merged_df[required_features].notna().all(axis=1)].copy()

if len(df_with_features) == 0:
    # Use courses with at least study_hours
    df_with_features = merged_df[merged_df['study_hours'].notna()].copy()

# Create composite difficulty score
# Normalize each feature to [0, 1] range, then combine

# Initialize difficulty components
difficulty_components = {}

# 1. Study hours: Higher = more difficult (normalize to [0,1])
if 'study_hours' in df_with_features.columns and df_with_features['study_hours'].notna().sum() > 0:
    hours_min = df_with_features['study_hours'].min()
    hours_max = df_with_features['study_hours'].max()
    if hours_max > hours_min:
        difficulty_components['study_hours'] = (df_with_features['study_hours'] - hours_min) / (hours_max - hours_min)
    else:
        difficulty_components['study_hours'] = pd.Series(0.5, index=df_with_features.index)

# 2. Learning average: Lower = more difficult (invert and normalize)
if 'learning_avg' in df_with_features.columns and df_with_features['learning_avg'].notna().sum() > 0:
    learning_min = df_with_features['learning_avg'].min()
    learning_max = df_with_features['learning_avg'].max()
    if learning_max > learning_min:
        # Invert: lower learning_avg -> higher difficulty score
        difficulty_components['learning_avg'] = 1 - ((df_with_features['learning_avg'] - learning_min) / (learning_max - learning_min))
    else:
        difficulty_components['learning_avg'] = pd.Series(0.5, index=df_with_features.index)

# 3. Structure average: Lower = more difficult (invert and normalize)
if 'structure_avg' in df_with_features.columns and df_with_features['structure_avg'].notna().sum() > 0:
    structure_min = df_with_features['structure_avg'].min()
    structure_max = df_with_features['structure_avg'].max()
    if structure_max > structure_min:
        # Invert: lower structure_avg -> higher difficulty score
        difficulty_components['structure_avg'] = 1 - ((df_with_features['structure_avg'] - structure_min) / (structure_max - structure_min))
    else:
        difficulty_components['structure_avg'] = pd.Series(0.5, index=df_with_features.index)

# 4. Environment average: Lower = more difficult (invert and normalize)
if 'environment_avg' in df_with_features.columns and df_with_features['environment_avg'].notna().sum() > 0:
    env_min = df_with_features['environment_avg'].min()
    env_max = df_with_features['environment_avg'].max()
    if env_max > env_min:
        # Invert: lower environment_avg -> higher difficulty score
        difficulty_components['environment_avg'] = 1 - ((df_with_features['environment_avg'] - env_min) / (env_max - env_min))
    else:
        difficulty_components['environment_avg'] = pd.Series(0.5, index=df_with_features.index)

# Combine all components into composite difficulty score (equal weights)
if len(difficulty_components) > 0:
    # Average all available components
    composite_scores = pd.DataFrame(difficulty_components).mean(axis=1)
    df_with_features['difficulty_score'] = composite_scores
    
    
    # Calculate median of composite score
    median_score = composite_scores.median()
    
    # Define difficulty labels based on median split of composite score
    # Difficult = 1 (composite score >= median)
    # Easy = 0 (composite score < median)
    df_with_features['difficulty_manual'] = (df_with_features['difficulty_score'] >= median_score).astype(int)
    
    # Create individual difficulty labels for each feature based on median split
    
    # 1. Study hours: Higher = more difficult
    if 'study_hours' in df_with_features.columns and df_with_features['study_hours'].notna().sum() > 0:
        median_hours = df_with_features['study_hours'].median()
        df_with_features['difficulty_study_hours'] = (df_with_features['study_hours'] >= median_hours).astype(int)
    
    # 2. Average grade: Lower = more difficult (invert)
    if 'avg_grade' in df_with_features.columns and df_with_features['avg_grade'].notna().sum() > 0:
        median_grade = df_with_features['avg_grade'].median()
        df_with_features['difficulty_avg_grade'] = (df_with_features['avg_grade'] < median_grade).astype(int)
    
    # 3. Learning average: Lower = more difficult (invert)
    if 'learning_avg' in df_with_features.columns and df_with_features['learning_avg'].notna().sum() > 0:
        median_learning = df_with_features['learning_avg'].median()
        df_with_features['difficulty_learning_avg'] = (df_with_features['learning_avg'] < median_learning).astype(int)
    
    # 4. Structure average: Lower = more difficult (invert)
    if 'structure_avg' in df_with_features.columns and df_with_features['structure_avg'].notna().sum() > 0:
        median_structure = df_with_features['structure_avg'].median()
        df_with_features['difficulty_structure_avg'] = (df_with_features['structure_avg'] < median_structure).astype(int)
    
    # 5. Environment average: Lower = more difficult (invert)
    if 'environment_avg' in df_with_features.columns and df_with_features['environment_avg'].notna().sum() > 0:
        median_environment = df_with_features['environment_avg'].median()
        df_with_features['difficulty_environment_avg'] = (df_with_features['environment_avg'] < median_environment).astype(int)
    
else:
    df_with_features['difficulty_score'] = 0.5
    df_with_features['difficulty_manual'] = 0


# Show statistics by difficulty category

# Build aggregation dictionary with available features
agg_dict = {
    'difficulty_score': ['mean', 'std', 'min', 'max'],
    'course_number': ['mean', 'std', 'min', 'max']
}

# Add features if they exist
if 'study_hours' in df_with_features.columns:
    agg_dict['study_hours'] = ['mean', 'std', 'min', 'max']
if 'avg_grade' in df_with_features.columns:
    agg_dict['avg_grade'] = ['mean', 'std']
if 'learning_avg' in df_with_features.columns:
    agg_dict['learning_avg'] = ['mean', 'std']
if 'structure_avg' in df_with_features.columns:
    agg_dict['structure_avg'] = ['mean', 'std']
if 'environment_avg' in df_with_features.columns:
    agg_dict['environment_avg'] = ['mean', 'std']

difficulty_stats = df_with_features.groupby('difficulty_manual').agg(agg_dict).round(4)

# Visualize the distribution
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Histogram of composite difficulty score by difficulty label
if 'difficulty_score' in df_with_features.columns:
    axes[0, 0].hist(df_with_features[df_with_features['difficulty_manual']==0]['difficulty_score'], 
                    bins=20, alpha=0.7, label='Easy (0)', color='green')
    axes[0, 0].hist(df_with_features[df_with_features['difficulty_manual']==1]['difficulty_score'], 
                    bins=20, alpha=0.7, label='Difficult (1)', color='red')
    if 'difficulty_score' in df_with_features.columns:
        median_score = df_with_features['difficulty_score'].median()
        axes[0, 0].axvline(median_score, color='black', linestyle='--', linewidth=2, 
                          label=f'Median ({median_score:.3f})')
    axes[0, 0].set_xlabel('Composite Difficulty Score')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].set_title('Distribution of Composite Difficulty Score')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

# 2. Box plot comparing composite difficulty score
if 'difficulty_score' in df_with_features.columns:
    box_data = [
        df_with_features[df_with_features['difficulty_manual']==0]['difficulty_score'].values,
        df_with_features[df_with_features['difficulty_manual']==1]['difficulty_score'].values
    ]
    axes[0, 1].boxplot(box_data, labels=['Easy (0)', 'Difficult (1)'])
    axes[0, 1].set_ylabel('Composite Difficulty Score')
    axes[0, 1].set_title('Composite Difficulty Score by Category')
    axes[0, 1].grid(True, alpha=0.3)

# 3. Study hours by difficulty (if available)
if 'study_hours' in df_with_features.columns:
    axes[1, 0].hist(df_with_features[df_with_features['difficulty_manual']==0]['study_hours'], 
                    bins=20, alpha=0.7, label='Easy (0)', color='green')
    axes[1, 0].hist(df_with_features[df_with_features['difficulty_manual']==1]['study_hours'], 
                    bins=20, alpha=0.7, label='Difficult (1)', color='red')
    axes[1, 0].set_xlabel('Study Hours per Week')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].set_title('Study Hours Distribution by Difficulty Label')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

# 4. Component contributions visualization
if len(difficulty_components) > 0:
    component_names = list(difficulty_components.keys())
    easy_means = [df_with_features[df_with_features['difficulty_manual']==0][f'difficulty_score'].mean() 
                  if 'difficulty_score' in df_with_features.columns else 0.5]
    difficult_means = [df_with_features[df_with_features['difficulty_manual']==1][f'difficulty_score'].mean() 
                       if 'difficulty_score' in df_with_features.columns else 0.5]
    
    # Show average component values by difficulty
    if 'study_hours' in df_with_features.columns:
        easy_hours = df_with_features[df_with_features['difficulty_manual']==0]['study_hours'].mean()
        diff_hours = df_with_features[df_with_features['difficulty_manual']==1]['study_hours'].mean()
        axes[1, 1].barh(['Easy', 'Difficult'], [easy_hours, diff_hours], 
                       color=['green', 'red'], alpha=0.7)
        axes[1, 1].set_xlabel('Average Study Hours')
        axes[1, 1].set_title('Average Study Hours by Difficulty Category')
        axes[1, 1].grid(True, alpha=0.3, axis='x')
    else:
        axes[1, 1].text(0.5, 0.5, 'Study hours data\nnot available', 
                       ha='center', va='center', transform=axes[1, 1].transAxes)
        axes[1, 1].set_title('Component Analysis')

plt.tight_layout()
plt.savefig('difficulty_manual_definition.png', dpi=300, bbox_inches='tight')
plt.show()


difficulty_cols = [col for col in df_with_features.columns if col.startswith('difficulty_')]
for col in sorted(difficulty_cols):
    if col not in ['difficulty_score', 'difficulty_manual']:
        non_null = df_with_features[col].notna().sum()



## Step 5: Merge Utilization Rate and Run T-Tests

We will:
1. Load utilization_rate data from WebReg
2. Merge with our difficulty-labeled dataset
3. Run independent samples t-tests comparing utilization_rate between Easy (0) and Difficult (1) for each difficulty feature


In [ ]:
# Step 5: Merge utilization_rate and run t-tests

# Load WebReg data with utilization_rate
webreg_data = pd.read_csv('webreg_data/results/webreg_processed_data.csv')

# Prepare WebReg data for merging
# Extract course number from course column (format: dsc_100 -> 100)
def extract_course_num_from_webreg(course_str):
    """Extract course number from webreg course format (e.g., 'dsc_100' -> 100)"""
    if pd.isna(course_str):
        return None
    import re
    match = re.search(r'dsc[_\s]*(\d+)', str(course_str).lower())
    if match:
        return int(match.group(1))
    return None

webreg_data['course_number'] = webreg_data['course'].apply(extract_course_num_from_webreg)
webreg_data['quarter'] = webreg_data['quarter'].str.lower()

# Select only the columns we need
webreg_merge = webreg_data[['course_number', 'quarter', 'utilization_rate']].copy()

# Merge with df_with_features
df_with_util = df_with_features.merge(
    webreg_merge,
    on=['course_number', 'quarter'],
    how='inner'
)


# Get all difficulty feature columns
difficulty_features = [col for col in df_with_util.columns if col.startswith('difficulty_') and col != 'difficulty_score']

# Run independent samples t-tests for each difficulty feature

t_test_results = []

for feature in difficulty_features:
    if feature not in df_with_util.columns:
        continue
    
    # Get groups
    easy_group = df_with_util[df_with_util[feature] == 0]['utilization_rate'].dropna()
    difficult_group = df_with_util[df_with_util[feature] == 1]['utilization_rate'].dropna()
    
    if len(easy_group) == 0 or len(difficult_group) == 0:
        continue
    
    # Calculate means
    easy_mean = easy_group.mean()
    difficult_mean = difficult_group.mean()
    mean_diff = abs(difficult_mean - easy_mean)
    
    # Run t-test
    t_stat, p_value = ttest_ind(easy_group, difficult_group)
    
    # Store results
    t_test_results.append({
        'Difficulty_Feature': feature,
        'Easy_Mean': easy_mean,
        'Difficult_Mean': difficult_mean,
        'Mean_Difference': mean_diff,
        't_statistic': abs(t_stat),  # Use absolute value to match absolute mean difference
        'p_value': p_value,
        'n_Easy': len(easy_group),
        'n_Difficult': len(difficult_group),
        'Significant': 'Yes' if p_value < 0.05 else 'No'
    })
    

# Create results DataFrame
t_test_df = pd.DataFrame(t_test_results)


# Visualize results
if len(t_test_results) > 0:
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    for idx, feature in enumerate(difficulty_features[:6]):  # Plot up to 6 features
        if feature not in df_with_util.columns:
            continue
        
        easy_group = df_with_util[df_with_util[feature] == 0]['utilization_rate'].dropna()
        difficult_group = df_with_util[df_with_util[feature] == 1]['utilization_rate'].dropna()
        
        if len(easy_group) == 0 or len(difficult_group) == 0:
            continue
        
        # Box plot
        box_data = [easy_group.values, difficult_group.values]
        bp = axes[idx].boxplot(box_data, labels=['Easy (0)', 'Difficult (1)'], patch_artist=True)
        bp['boxes'][0].set_facecolor('green')
        bp['boxes'][0].set_alpha(0.7)
        bp['boxes'][1].set_facecolor('red')
        bp['boxes'][1].set_alpha(0.7)
        
        axes[idx].set_ylabel('Utilization Rate (%)')
        axes[idx].set_title(f'{feature}\np={t_test_df[t_test_df["Difficulty_Feature"]==feature]["p_value"].values[0]:.4f}')
        axes[idx].grid(True, alpha=0.3)
    
    # Hide unused subplots
    for idx in range(len(difficulty_features), 6):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.savefig('t_test_utilization_by_difficulty.png', dpi=300, bbox_inches='tight')
    plt.show()



## Step 6: Two-Way ANOVA Tests

We will run two-way ANOVA tests to examine interaction effects between:
- **Composite difficulty score** (`difficulty_manual`) × **Individual features** (study_hours, avg_grade, learning_avg, structure_avg, environment_avg)

**Purpose**: Test if the relationship between each feature and utilization_rate differs significantly between Easy (0) and Difficult (1) courses.

**Hypotheses**:
- **H₀**: No interaction effect (the relationship between feature and utilization_rate is the same for Easy and Difficult courses)
- **H₁**: Significant interaction effect (the relationship differs between Easy and Difficult courses)


In [ ]:
# Step 6: Two-way ANOVA tests

# Features to test interactions with
interaction_features = ['study_hours', 'avg_grade', 'learning_avg', 'structure_avg', 'environment_avg']

anova_results = []

for feature in interaction_features:
    if feature not in df_with_util.columns:
        continue
    
    # Filter to rows with all required data
    test_data = df_with_util[[
        'difficulty_manual', 
        feature, 
        'utilization_rate'
    ]].dropna()
    
    if len(test_data) < 10:  # Need minimum data
        continue
    
    # Convert difficulty_manual to categorical for ANOVA
    test_data['difficulty_manual'] = test_data['difficulty_manual'].astype('category')
    
    # Create interaction term formula
    # Format: utilization_rate ~ C(difficulty_manual) * feature
    formula = f'utilization_rate ~ C(difficulty_manual) * {feature}'
    
    try:
        # Fit two-way ANOVA model
        model = ols(formula, data=test_data).fit()
        anova_table = anova_lm(model, typ=2)
        
        # Extract interaction p-value
        interaction_row = anova_table.loc[f'C(difficulty_manual):{feature}']
        interaction_p = interaction_row['PR(>F)']
        
        # Extract main effects
        main_effect_diff = anova_table.loc['C(difficulty_manual)', 'PR(>F)']
        main_effect_feature = anova_table.loc[feature, 'PR(>F)']
        
        # Calculate means for each group
        means = test_data.groupby('difficulty_manual').agg({
            feature: 'mean',
            'utilization_rate': 'mean'
        })
        
        # Store results
        anova_results.append({
            'Feature': feature,
            'Interaction_p_value': interaction_p,
            'Main_Effect_Difficulty_p': main_effect_diff,
            'Main_Effect_Feature_p': main_effect_feature,
            'Easy_Mean_Feature': means.loc[0, feature],
            'Difficult_Mean_Feature': means.loc[1, feature],
            'Easy_Mean_Utilization': means.loc[0, 'utilization_rate'],
            'Difficult_Mean_Utilization': means.loc[1, 'utilization_rate'],
            'n': len(test_data),
            'Interaction_Significant': 'Yes' if interaction_p < 0.05 else 'No'
        })
        
    except Exception as e:
        continue

# Create results DataFrame
anova_df = pd.DataFrame(anova_results)

if len(anova_df) > 0:
    